## Battle of Neighbourhoods : Toronto


### Preparing the Dataframe from Wiki
    1)	Using beautiful soup package and lxml library, we must scrape the list of postal code of Toronto from Wikipedia.
    2)	Once the data is scrapped, we must format it by cleaning the ‘Not assigned’ values of Boroughs and also if any Neighbourhoods of the borough has ‘Not assigned’ value, we must update it with the borough value.
    3) Also, we must make sure there is one unique row for each postal code. For this, we will be joining the Neighbourhood data for each postal code. Below is the image of the formatted data frame that we have obtained from the raw data of Wikipedia.


In [1]:
#Toronto Neighbourhood Dataframe from Part1 of the assignment

get_ipython().system(u'pip install --upgrade beautifulsoup4')
get_ipython().system(u'pip install --upgrade lxml')
get_ipython().system(u'pip install --upgrade html5lib')
get_ipython().system(u'pip install --upgrade requests')

from bs4 import BeautifulSoup
import requests
import pandas as pd

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source,'lxml')

Toronto_table = soup.find('table',{'class':'wikitable sortable'})

PostalCodeList=[]
for i in Toronto_table.findAll('tr'):
    split_rows=i.text.split('\n')
    PostalCodeList.append(split_rows)
    empty_string=''
    while empty_string in split_rows:
        split_rows.remove(empty_string) 

PostalCodeDf=pd.DataFrame(PostalCodeList[1:], columns=PostalCodeList[0])

FormattedDf=PostalCodeDf[PostalCodeDf.Borough!='Not assigned']

for i in range(0,len(FormattedDf)):
    if FormattedDf.iloc[i]['Neighbourhood']=='Not assigned':
        FormattedDf.iloc[i]['Neighbourhood']=FormattedDf.iloc[i]['Borough']
FormattedDf=FormattedDf.reset_index()
FormattedDf=FormattedDf.drop(['index'],axis=1)
GroupedDf=pd.DataFrame(FormattedDf.groupby('Postcode',as_index=False).agg(lambda x:', '.join(x['Neighbourhood'])))

FinalDf=pd.merge(GroupedDf, FormattedDf, left_on='Postcode', right_on='Postcode', how='left').drop(['Borough_x','Neighbourhood_y'], axis=1)

FinalDf=FinalDf.drop_duplicates(subset=['Postcode','Borough_y','Neighbourhood_x'])

FinalDf=FinalDf[['Postcode','Borough_y','Neighbourhood_x']]

FinalDf.columns=['Postcode','Borough','Neighbourhood']

FinalDf=FinalDf.reset_index()
FinalDf=FinalDf.drop(['index'],axis=1)
FinalDf.shape

    100% |████████████████████████████████| 102kB 709kB/s a 0:00:01
  Found existing installation: beautifulsoup4 4.6.0
    Uninstalling beautifulsoup4-4.6.0:
      Successfully uninstalled beautifulsoup4-4.6.0
    100% |████████████████████████████████| 5.6MB 129kB/s ta 0:00:011    90% |█████████████████████████████   | 5.1MB 531kB/s eta 0:00:02    92% |█████████████████████████████▊  | 5.2MB 641kB/s eta 0:00:01
  Found existing installation: lxml 4.1.0
    Uninstalling lxml-4.1.0:
      Successfully uninstalled lxml-4.1.0
    100% |████████████████████████████████| 122kB 709kB/s ta 0:00:01
Requirement not upgraded as not directly required: webencodings in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)
Requirement not upgraded as not directly required: six>=1.9 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)
  Found existing installation: html5lib 0.999999999
    Uninstalling html5lib-0.999999999:
      Successfully uninstalled htm

(103, 3)

### Reading the cooridnates data from csv

In [2]:
import pandas as pd 

lat_long=pd.read_csv('http://cocl.us/Geospatial_data') 
lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Appending the coordinates from geolocation dataframe to final dataframe
        Now, we must append the coordinates for each borough with the above data frame to proceed with our analysis. Coordinates details are obtained from Geospatial Data csv file. After appending, a sample of the final data frame looks like below,

In [3]:
FinalDf['Latitude']=lat_long['Latitude']
FinalDf['Longitude']=lat_long['Longitude']
FinalDf.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Shape of the final dataframe

In [4]:
FinalDf.shape

(103, 5)

### Importing required libraries for Data Visualization and Machine Learning

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  21.13 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  31.44 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  46.11 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  29.30 MB/s
vincent-0.4.4- 100% |###################

### Getting Geographical coordinates of Toronto city

In [6]:
address = 'Toronto, Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of Toronto City are 43.653963, -79.387207.


### Creating map of Toronto using latitude and longitude values

In [7]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

### Adding markers to map
        Once the data analysis is over and the data is prepared, we will be proceeding to visualization part to understand in a better way. Below is the map of Toronto with all the postal codes marked.

In [8]:
for lat, lng, borough, neighborhood in zip(FinalDf['Latitude'], FinalDf['Longitude'], FinalDf['Borough'], FinalDf['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Picking Boroughs that contain the string 'Scarborough'
        Now, for analysis, we are taking only Scarborough’s neighbourhoods as the Tamil population is more here. 

In [9]:
Scarborough_data = FinalDf[FinalDf['Borough'].str.contains('Scarborough')].reset_index(drop=True)
Scarborough_data = Scarborough_data[:16]
Scarborough_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [10]:
Scarborough_data.shape

(16, 5)

### Creating map of Toronto using latitude and longitude values and adding markers
        Below is the map of the Scarborough neighbourhoods.

In [11]:
map_scar = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scar)  
    
map_scar

### Getting location information from Foursquare
       We have decided the neighbourhood, we must analyse the neighbourhoods by knowing the most common venues around the each neighbourhoods. For this, we will be using the FourSquare API, where providing the neighbourhood coordinates along with the radius, we will be getting the Venue details for all the neighbourhoods in Scarborough. Below is the same of the data frame after the venue extraction and appending it to our     neighbourhood data frame.

In [12]:
Scarborough_data.loc[0, 'Neighbourhood']

neighborhood_latitude = Scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Scarborough_data.loc[0, 'Neighbourhood'] # neighborhood name

LIMIT = 100
radius = 500
CLIENT_ID = 'MVWU0ZFMUZKRGMN2MCAZRLKE4FTJNKZLQO1FMCX4O0ADKDJK' # your Foursquare ID
CLIENT_SECRET = 'HQ00ZDG3LAYOJHLSKDIU1RUKPSV3OKDPYDEQC5BCNUNMBQ3P' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c3419106a60713c264c2159'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bb6b9446edc76b0d771311c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16e941735',
         'name': 'Fast Food Restaurant',
         'pluralName': 'Fast Food Restaurants',
         'primary': True,
         'shortName': 'Fast Food'}],
       'id': '4bb6b9446edc76b0d771311c',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside & Sheppard',
        'distance': 387,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80744841934756,
          'ln

### Function that extracts the category of the venue

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

### Filtering and cleaning columns based on category

In [14]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


### Getting nearby venues for each neighbourhood

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighbourhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )

print(Scarborough_venues.shape)
Scarborough_venues.head()

(84, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


### Checking the number of unique categories of venues

In [16]:
Scarborough_venues.groupby('Neighbourhood').count()

print('There are {} uniques categories.'.format(len(Scarborough_venues['Venue Category'].unique())))

There are 50 uniques categories.


### One hot encoding the venue categories, so that it's easy to cluster the data later
       We have received the necessary data, but this is not in the expected format to apply ML algorithms. We must further format it. As we are     dealing with categorical data (Venue categories), we will have to group these data based on OneHot encoding method for each neighbourhoods. After grouping, the sample of the data frame looks like below.

In [17]:
# one hot encoding
Scarborough_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scarborough_onehot['Neighbourhood'] = Scarborough_venues['Neighbourhood'] 

### Moving neighborhood column to the first column

In [18]:
fixed_columns = [Scarborough_onehot.columns[-1]] + list(Scarborough_onehot.columns[:-1])
Scarborough_onehot = Scarborough_onehot[fixed_columns]

Scarborough_onehot.head()

,Neighbourhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Thai Restaurant,Train Station,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


### Grouping the one hot dataframe based on Neighborhood

In [19]:
Scarborough_grouped = Scarborough_onehot.groupby('Neighbourhood').mean().reset_index()
Scarborough_grouped.head()

,Neighbourhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.0,0.000,0.000,0.0,0.333333,0.0,0.0,0.00,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.00,0.0,0.0,0.000,0.0,0.0
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000,0.0,0.000,0.000,0.0,0.000000,0.0,0.0,0.00,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.000000,0.0,0.00,0.0,0.0,0.000,0.0,0.0
2,"Birch Cliff, Cliffside West",0.0,0.000,0.0,0.000,0.000,0.0,0.000000,0.0,0.0,0.25,0.000,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.25,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.25,0.0,0.0,0.000,0.0,0.0
3,Cedarbrae,0.0,0.125,0.0,0.125,0.125,0.0,0.000000,0.0,0.0,0.00,0.125,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.00,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.125000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.125,0.0,0.0
4,"Clairlea, Golden Mile, Oakridge",0.0,0.000,0.0,0.200,0.000,0.0,0.000000,0.2,0.1,0.00,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.1,0.000,0.0,0.00,0.0,0.000,0.0,0.1,0.0,0.0,0.0,0.0,0.000000,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.1,0.0,0.000,0.0,0.0


### Finding the most occurences of each category in the respective Neighbourhood 
        To understand the most occurring frequency of a particular venue of a Neighbourhood, we can generate top 5 venues for a neighbourhood.

In [20]:
num_top_venues = 5
for hood in Scarborough_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Scarborough_grouped[Scarborough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

----Agincourt----
                 venue  freq
0               Lounge  0.33
1       Breakfast Spot  0.33
2       Sandwich Place  0.33
3  American Restaurant  0.00
4            Pet Store  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0           Playground   0.5
1                 Park   0.5
2  American Restaurant   0.0
3            Pet Store   0.0
4  Japanese Restaurant   0.0


----Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1  General Entertainment  0.25
2           Skating Rink  0.25
3                   Café  0.25
4               Pharmacy  0.00


----Cedarbrae----
                venue  freq
0              Lounge  0.12
1              Bakery  0.12
2                Bank  0.12
3     Thai Restaurant  0.12
4  Athletics & Sports  0.12


----Clairlea, Golden Mile, Oakridge----
          venue  freq
0        Bakery   0.2
1      Bus Line   0.2
2  Intersection   0.1
3  Soccer Field   0.1
4  

### Creating a new dataframe based on Most common venues in each Neighbourhood
        Similarly, we can get the Top 10 venues for each neighbourhoods and can for a dataframe, which will be used for applying ML algorithms.

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Scarborough_grouped['Neighbourhood']

for ind in np.arange(Scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Sandwich Place,Breakfast Spot,Lounge,Vietnamese Restaurant,Coffee Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Chinese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Lounge,Caribbean Restaurant,Hakka Restaurant,Vietnamese Restaurant,Cosmetics Shop
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Park,Intersection,Soccer Field,Fast Food Restaurant,Bus Station,Metro Station,College Stadium,Furniture / Home Store
5,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Noodle House,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Shopping Mall,Italian Restaurant,Rental Car Location,Chinese Restaurant,Caribbean Restaurant
6,"Cliffcrest, Cliffside, Scarborough Village West",American Restaurant,Motel,Coffee Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Pet Store,Chinese Restaurant,Furniture / Home Store,Latin American Restaurant,Vietnamese Restaurant,Shopping Mall,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant
8,"East Birchmount Park, Ionview, Kennedy Park",Discount Store,Bus Station,Department Store,Train Station,Coffee Shop,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Furniture / Home Store
9,"Guildwood, Morningside, West Hill",Mexican Restaurant,Spa,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant


### Clustering the Neighbourhoods using K-Means
        We will be using the K-Means clustering ML algorithm to cluster the neighbourhoods of Scarborough which will be helpful to infer the suitable neighbourhood to open a restaurant. Below is the sample of the dataframe with the Cluster Labels after applying the ML algorithm.

In [22]:
# set number of clusters
kclusters = 3

Scarborough_grouped_clustering = Scarborough_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

### Merging the grouped data to add latitude/longitude for each neighborhood

In [23]:
Scarborough_merged = Scarborough_data

# add clustering labels
Scarborough_merged['Cluster Labels'] = kmeans.labels_

Scarborough_merged = Scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Scarborough_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,1,Bar,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Mexican Restaurant,Spa,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Lounge,Caribbean Restaurant,Hakka Restaurant,Vietnamese Restaurant,Cosmetics Shop
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0,Playground,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,0,Discount Store,Bus Station,Department Store,Train Station,Coffee Shop,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Furniture / Home Store
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,0,Bakery,Bus Line,Park,Intersection,Soccer Field,Fast Food Restaurant,Bus Station,Metro Station,College Stadium,Furniture / Home Store
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,0,American Restaurant,Motel,Coffee Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


### Creating color based clustered map and adding markers
        Plotting the clustered neighbourhood in the Toronto map, we will be getting the below. Each colour represents each cluster. Here, we have used 3 Clusters as the neighbourhoods are very less.

In [24]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Scarborough_merged['Latitude'], Scarborough_merged['Longitude'], Scarborough_merged['Neighbourhood'], Scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Top 10 venues of first cluster

In [25]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 0, Scarborough_merged.columns[[2] + list(range(5, Scarborough_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",0,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
2,"Guildwood, Morningside, West Hill",0,Mexican Restaurant,Spa,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant
3,Woburn,0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
4,Cedarbrae,0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Lounge,Caribbean Restaurant,Hakka Restaurant,Vietnamese Restaurant,Cosmetics Shop
5,Scarborough Village,0,Playground,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
6,"East Birchmount Park, Ionview, Kennedy Park",0,Discount Store,Bus Station,Department Store,Train Station,Coffee Shop,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Furniture / Home Store
7,"Clairlea, Golden Mile, Oakridge",0,Bakery,Bus Line,Park,Intersection,Soccer Field,Fast Food Restaurant,Bus Station,Metro Station,College Stadium,Furniture / Home Store
8,"Cliffcrest, Cliffside, Scarborough Village West",0,American Restaurant,Motel,Coffee Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
9,"Birch Cliff, Cliffside West",0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
11,"Maryvale, Wexford",0,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,College Stadium,General Entertainment,Furniture / Home Store,Fried Chicken Joint


### Top 10 venues of second cluster

In [26]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 1, Scarborough_merged.columns[[2] + list(range(5, Scarborough_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek, Rouge Hill, Port Union",1,Bar,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
14,"Agincourt North, L'Amoreaux East, Milliken, St...",1,Park,Playground,Chinese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


### Top 10 venues of third cluster

In [27]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 2, Scarborough_merged.columns[[2] + list(range(5, Scarborough_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"Dorset Park, Scarborough Town Centre, Wexford ...",2,Indian Restaurant,Pet Store,Chinese Restaurant,Furniture / Home Store,Latin American Restaurant,Vietnamese Restaurant,Shopping Mall,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant


## Inference:

#### Based on the above clustering of the neighbourhoods, one can decide where to open a coffee shop based on their preferences of the top 10 venues from each neighbourhoods.